In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import feather
import nltk

In [2]:
dat = pickle.load(open("./grouped_sample.pkl", 'rb'))

In [3]:
dat.shape

(681819, 22)

In [4]:
dat.head()

,video_id,duration,title,channel_id,channel_title,published_at,views,likes,dislikes,comments,...,1,2,3,4,5,6,7,8,9,10
0,--DwgB78t-c,00:02:36,Fruto Dulce - Geraldine Rojas & Ezequiel Paludi 3,UCdzU3DSGzyWzN2118yd9X9g,Abdullah AYDIN,2012-01-19 18:20:02,1671,10,1,0,...,0,1,0,0,0,0,0,0,0,0
1,--NZRkXBV7k,00:04:53,Carlos Santana presents Cindy Blackman in Mexi...,UC0UnhAG47DRyVZGVcbhAXhQ,Saul TP,2015-03-30 04:04:40,9994,41,3,2,...,0,1,0,0,0,0,0,0,0,0
2,--hoQ2sGG4M,00:04:35,"Taoist monk Tian Xin Shan in Brazil,Wudang San...",UCXjtAvK5P3wXBGh0vbGylzg,TheTVtaoista,2009-08-07 06:36:33,3407,8,1,2,...,0,0,0,0,0,0,0,0,0,0
3,--sBoaqBlzA,00:08:19,"MAKEUP FOR GUYS: Actors, TV Hosts, On Camera ...",UCeKHMeUlcLNPLCLUfZUQI2w,Tiffany Hendra,2011-08-04 00:47:49,15816,120,2,8,...,0,1,0,1,0,0,0,0,0,0
4,--7h1S4neDM,00:03:19,Pet Rescue Saga Level 539 2 Stars No Boosts,UCNWPDyaWf2eAHnofFLSnEMg,All Gamers,2014-04-29 15:44:44,2167,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [8]:
# stops = set(stopwords.words('english')) # could consider using combined stop words from most popular languages
stops = set(stopwords.words())

In [5]:
dat['title'].head()

0    Fruto Dulce - Geraldine Rojas & Ezequiel Paludi 3
1    Carlos Santana presents Cindy Blackman in Mexi...
2    Taoist monk Tian Xin Shan in Brazil,Wudang San...
3    MAKEUP FOR GUYS:  Actors, TV Hosts, On Camera ...
4          Pet Rescue Saga Level 539 2 Stars No Boosts
Name: title, dtype: object

In [15]:
# tokens = [word_tokenize(t) for t in dat['title'] if t is not None]
from IPython import display
tokens = []
w = 0
for t in dat['title']:
    if t is None:
        tokens += [[]]
    else:
        tokens += [word_tokenize(t)]
    display.clear_output(wait=True)
    w += 1
    print("at sentence {0}".format(w))    

at sentense 681819


In [17]:
tokens[:5]

[['Fruto', 'Dulce', '-', 'Geraldine', 'Rojas', '&', 'Ezequiel', 'Paludi', '3'],
 ['Carlos',
  'Santana',
  'presents',
  'Cindy',
  'Blackman',
  'in',
  'Mexico',
  'City'],
 ['Taoist',
  'monk',
  'Tian',
  'Xin',
  'Shan',
  'in',
  'Brazil',
  ',',
  'Wudang',
  'Sanfeng',
  "T'ai",
  'Chi',
  'exercises'],
 ['MAKEUP',
  'FOR',
  'GUYS',
  ':',
  'Actors',
  ',',
  'TV',
  'Hosts',
  ',',
  'On',
  'Camera',
  'Experts'],
 ['Pet', 'Rescue', 'Saga', 'Level', '539', '2', 'Stars', 'No', 'Boosts']]

In [42]:
# pickle.dump(tokens, open("tokens.pkl", "wb"))

**Assuming most titles are in English (consider language detection later)**

In [9]:
tokens = pickle.load(open("tokens.pkl", "rb"))

In [10]:
import re

In [11]:
pure_tokens = [" ".join([w.lower() for w in sent if w.lower() not in stops and re.match("\w+", w)])\
               for sent in tokens]

In [12]:
pure_tokens[:5]

['fruto dulce geraldine rojas ezequiel paludi 3',
 'carlos santana presents cindy blackman mexico city',
 "taoist monk tian xin brazil wudang sanfeng t'ai exercises",
 'makeup guys actors tv hosts camera experts',
 'pet rescue saga level 539 2 stars boosts']

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)

In [21]:
features = vectorizer.fit_transform(pure_tokens)

In [22]:
feature_names = vectorizer.get_feature_names()

In [23]:
# feat_mat = features.todense()

In [24]:
features.shape

(681819, 5000)

In [25]:
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression

In [26]:
fs, ps = f_regression(features, np.log(dat['views']), center=False)
alpha = 0.01
sig_inds = np.where(ps < alpha)[0]
reg = LinearRegression(n_jobs=4)
reg.fit(features, np.log(dat['views']))
views_stat = pd.DataFrame({"feature_id": sig_inds,
              "word": np.array(feature_names)[sig_inds],
              "coef": reg.coef_[sig_inds], 
              "p": ps[sig_inds]}).sort_values('coef', ascending=False)[['word', 'feature_id', 'coef', 'p']]

In [30]:
views_stat.head(10)

,word,feature_id,coef,p
3940,sevenfold,3940,1.420393,1.367308e-34
4925,детей,4925,1.360317,3.526555e-33
1665,faze,1665,1.320893,4.176124e-28
3657,reset,3657,1.108704,3.360949e-46
729,britain,729,1.099806,5.744635e-34
4950,серия,4950,1.037553,3.900010e-63
3929,sesame,3929,1.020937,7.879123e-32
4943,рецепт,4943,1.018323,2.094089e-35
4715,viña,4715,0.971714,3.637986e-29
4781,webisode,4781,0.943215,1.486629e-31


In [31]:
views_stat.tail(10)

,word,feature_id,coef,p
503,ballroom,503,-0.538840,5.534705e-22
125,3gp,125,-0.543457,1.927154e-42
385,apresentação,385,-0.553671,1.323539e-20
4033,skatepark,4033,-0.560497,1.355982e-40
3586,ranked,3586,-0.560828,2.837027e-35
2201,iglesia,2201,-0.561589,4.916255e-23
3990,showreel,3990,-0.564596,8.195069e-20
4171,ssbb,4171,-0.648883,3.861534e-21
1256,darkorbit,1256,-0.717136,7.847164e-32
460,avenged,460,-0.975518,6.304315e-35


In [32]:
fs, ps = f_regression(features, dat['duration'] / np.timedelta64(1, 's'), center=False)
alpha = 0.01
sig_inds = np.where(ps < alpha)[0]
reg = LinearRegression(n_jobs=4)
reg.fit(features, dat['duration'] / np.timedelta64(1, 's'))
duration_stat = pd.DataFrame({"feature_id": sig_inds,
              "word": np.array(feature_names)[sig_inds],
              "coef": reg.coef_[sig_inds], 
              "p": ps[sig_inds]}).sort_values('coef', ascending=False)[['word', 'feature_id', 'coef', 'p']]

In [33]:
duration_stat.head(10)

,word,feature_id,coef,p
2065,haul,2065,100.410777,5.237096e-322
3617,recensione,3617,80.550428,4.063264e-42
2829,metallica,2829,79.620075,1.088874e-135
4995,神魔之塔,4995,75.879521,7.527500e-40
3251,partie,3251,68.749905,1.169152e-67
3248,part3,3248,67.428301,2.930866e-40
3379,playthrough,3379,65.521045,1.985373e-139
3204,p1,3204,64.910025,1.570365e-37
1285,deel,1285,64.906273,9.453681e-39
3940,sevenfold,3940,63.532622,4.177819e-45


In [34]:
duration_stat.tail(10)

,word,feature_id,coef,p
4133,sparta,4133,-48.576669,1.969519e-15
4191,stats,4191,-48.724739,1.567058e-27
511,bande,511,-49.127083,2.288318e-20
943,cheerleaders,943,-50.351524,1.929454e-18
942,cheer,942,-50.567213,7.618557e-45
3067,nightcore,3067,-54.737429,7.784102e-21
4993,太鼓の達人,4993,-71.311398,5.095912e-12
4486,trailer,4486,-72.788387,0.000000e+00
4955,трейлер,4955,-97.373937,7.697384e-35
4541,tráiler,4541,-98.536315,2.168022e-10


### generate json list for word cloud

In [35]:
views_stat.shape

(5000, 4)

In [36]:
top100_views = views_stat.head(100)[['word', 'coef']]

In [37]:
top100_views['size'] = np.exp(5.5 * top100_views['coef']).astype(int) + 5
top100_views.rename(columns={"word": "text"}, inplace=True)
top100_views.head()

,text,coef,size
3940,sevenfold,1.420393,2475
4925,детей,1.360317,1780
1665,faze,1.320893,1434
3657,reset,1.108704,449
729,britain,1.099806,428


In [38]:
top100_views[['text', 'size']].to_json("word_cloud/top100_views_5000.json", orient='records')

In [39]:
bot100_views = views_stat.tail(100)[['word', 'coef']]
bot100_views['size'] = np.exp(-8.5 * bot100_views['coef']).astype(int) + 5
bot100_views.rename(columns={"word": "text"}, inplace=True)
bot100_views[['text', 'size']].to_json("word_cloud/bot100_views.json_5000", orient='records')

### TF-IDF

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, max_features=5000)
tdidf = vectorizer.fit_transform(pure_tokens)

In [41]:
words = vectorizer.get_feature_names()

In [42]:
words[250:260]

['advent',
 'adventure',
 'adventures',
 'advice',
 'ae',
 'aerial',
 'africa',
 'african',
 'afro',
 'aftermovie']

In [43]:
idf = vectorizer.idf_
idf

array([ 8.86417647,  8.25043607,  9.2850265 , ...,  9.75038975,
        8.94772404,  9.67034704])

In [44]:
tdidf

<681819x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2042418 stored elements in Compressed Sparse Row format>

In [45]:
fs, ps = f_regression(tdidf, np.log(dat['views']), center=False)
alpha = 0.01
sig_inds = np.where(ps < alpha)[0]
reg = LinearRegression(n_jobs=4)
reg.fit(tdidf, np.log(dat['views']))
views_stat_tdidf = pd.DataFrame({"feature_id": sig_inds,
              "word": np.array(words)[sig_inds],
              "coef": reg.coef_[sig_inds], 
              "p": ps[sig_inds]}).sort_values('coef', ascending=False)[['word', 'feature_id', 'coef', 'p']]

In [46]:
views_stat_tdidf.head(20)

,word,feature_id,coef,p
1665,faze,1665,2.593950,7.033313e-30
3286,peppa,3286,2.590707,6.906898e-45
3940,sevenfold,3940,2.353965,6.153159e-34
1387,doh,1387,2.059334,1.323623e-83
3657,reset,3657,2.004721,2.840415e-44
3929,sesame,3929,1.972696,3.716675e-31
729,britain,729,1.927408,6.926686e-32
4925,детей,4925,1.726540,1.175329e-32
4476,toys,4476,1.639863,2.235558e-143
4781,webisode,4781,1.631487,8.459493e-31


In [47]:
views_stat_tdidf.tail(20)

,word,feature_id,coef,p
3990,showreel,3990,-0.836485,1.191903e-18
3824,sale,3824,-0.839451,3.091472e-183
385,apresentação,385,-0.842155,2.320745e-19
2015,guild,2015,-0.849559,2.911795e-52
4284,sv,4284,-0.858576,6.261931e-25
4033,skatepark,4033,-0.861907,3.645241e-38
2231,information,2231,-0.866183,8.036755e-22
2546,lets,2546,-0.872722,2.241874e-59
3915,senior,3915,-0.881183,8.223449e-57
1993,grove,1993,-0.904690,6.217760e-21


### bigrams

In [48]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000,\
                             ngram_range = (2, 2))
features = vectorizer.fit_transform(pure_tokens)
feature_names = vectorizer.get_feature_names()
fs, ps = f_regression(features, np.log(dat['views']), center=False)
alpha = 0.01
sig_inds = np.where(ps < alpha)[0]
reg = LinearRegression(n_jobs=4)
reg.fit(features, np.log(dat['views']))
views_stat = pd.DataFrame({"feature_id": sig_inds,
              "word": np.array(feature_names)[sig_inds],
              "coef": reg.coef_[sig_inds], 
              "p": ps[sig_inds]}).sort_values('coef', ascending=False)[['word', 'feature_id', 'coef', 'p']]

In [49]:
views_stat.head(10)

,word,feature_id,coef,p
4979,уральские пельмени,4979,2.788098,3.571239e-11
1724,disney junior,1724,2.723160,1.521721e-18
2405,hard reset,2405,2.139729,2.767609e-19
3448,official mv,3448,2.018507,5.746141e-27
4392,swap force,4392,1.995135,2.234033e-08
4129,sing along,4129,1.920715,5.322569e-09
3240,mr bean,3240,1.882065,6.595934e-14
3200,moments episode,3200,1.881037,1.141150e-09
3232,movie clip,3232,1.817223,1.708964e-73
4998,인기가요 inkigayo,4998,1.794608,2.597464e-09


In [50]:
views_stat.tail(10)

,word,feature_id,coef,p
1925,eyed peas,1925,-1.338781,7.361969e-16
2781,last light,2781,-1.346087,1.844461e-06
4574,trans siberian,4574,-1.491705,3.037604e-07
882,albert hall,882,-1.498994,5.705677e-12
4950,zombies garden,4950,-1.560755,2.139907e-06
1372,chasse sous,1372,-1.655578,4.957017e-06
4942,yu gi,4942,-1.723673,1.959647e-43
1084,bath body,1084,-1.836075,1.468648e-06
4186,somebody used,4186,-1.864296,8.027923e-07
4144,skylanders swap,4144,-1.927097,4.283380e-07


### What about 3?

In [51]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000,\
                             ngram_range = (3, 3))
features = vectorizer.fit_transform(pure_tokens)
feature_names = vectorizer.get_feature_names()
fs, ps = f_regression(features, np.log(dat['views']), center=False)
alpha = 0.01
sig_inds = np.where(ps < alpha)[0]
reg = LinearRegression(n_jobs=4)
reg.fit(features, np.log(dat['views']))
views_stat = pd.DataFrame({"feature_id": sig_inds,
              "word": np.array(feature_names)[sig_inds],
              "coef": reg.coef_[sig_inds], 
              "p": ps[sig_inds]}).sort_values('coef', ascending=False)[['word', 'feature_id', 'coef', 'p']]

In [52]:
views_stat.head(10)

,word,feature_id,coef,p
2155,lol moments episode,2636,5.797829,1.739851e-05
3372,spotlight league legends,4101,5.596789,7.279373e-05
3491,surprise egg learn,4242,4.977492,4.427025e-05
1407,girls rainbow rocks,1741,4.605891,5.930941e-07
2880,play doh peppa,3499,4.313814,1.991457e-05
2234,mario 3d land,2736,4.259233,9.026767e-06
2665,official disney channel,3238,4.157216,1.896397e-04
2351,minecraft mob battles,2873,4.091725,1.326052e-04
231,3gs ipod touch,356,3.982495,1.034121e-04
3704,tv top 10,4495,3.905590,3.248346e-07


In [53]:
views_stat.tail(10)

,word,feature_id,coef,p
3790,vii advent children,4588,-2.552412,4.498069e-03
978,doh peppa pig,1240,-2.640899,1.865725e-04
766,clash clans town,985,-2.852679,5.329695e-06
4116,موقع نفهم موقع,4971,-3.192902,5.956471e-03
1052,egg learn word,1327,-3.310927,1.504307e-04
1083,equestria girls rainbow,1361,-3.503676,4.505173e-07
1421,go far kid,1757,-3.530142,5.936652e-04
3463,super mario 3d,4214,-3.688581,3.082788e-10
2223,man web shadows,2720,-3.971966,7.687022e-03
1797,iphone 3gs ipod,2216,-4.451688,2.731455e-03
